In [109]:
import requests
from functools import cache
import pandas as pd
from ratelimit import limits, sleep_and_retry
from tqdm import tqdm
from functools import partial
from datetime import datetime, timedelta
import pytz

TEAM_ID_FIELD = 'team_id'

def cache_limit_decorator(f):
    return cache(sleep_and_retry(limits(calls=55, period=60)(f)))


class OpenDotaApi:
    ROOT_URL = "https://api.opendota.com/api/"

    @cache
    def prepare_request(self, url):
        return partial(requests.get, f"{self.ROOT_URL}{url}")

    @cache_limit_decorator
    def get_teams(self):
        return requests.get(f"{self.ROOT_URL}teams").json()

    @cache_limit_decorator
    def get_team_players(self, team_id: int):
        return requests.get(f"{self.ROOT_URL}teams/{team_id}/players").json()

    @cache_limit_decorator
    def get_player_info(self, player_id: int):
        data = requests.get(f"{self.ROOT_URL}players/{player_id}").json()
        return {
            "account_id": player_id,
            "rank_tier": data["rank_tier"],
            "solo_mmr": data["mmr_estimate"]["estimate"],
            "leaderboard_rank": data["leaderboard_rank"],
        }

    @cache_limit_decorator
    def get_team_matches(self, team_id: int):
        req = self.prepare_request(f"teams/{team_id}/matches")
        return req().json()


open_doto_api = OpenDotaApi()


In [105]:
teams = open_doto_api.get_teams()
teams_df = pd.DataFrame.from_dict(teams)
teams_df["name"] = teams_df["name"].astype('str')
teams_df["tag"] = teams_df["tag"].astype('str')
teams_df["name"] = teams_df["name"].apply(lambda s: s.strip())

leaders_df = teams_df.sort_values(by=["rating"], ascending=False)[:150]
leaders_df.to_csv('data/leader_teams.csv', index=False)
leaders_df.head()

,team_id,rating,wins,losses,last_match_time,name,tag,logo_url
0,7119388,1550.29,216,173,1642705587,Team Spirit,TSpirit,https://steamusercontent-a.akamaihd.net/ugc/17...
1,7412785,1520.12,267,1,1639928575,CyberBonch-1,CB,https://steamusercontent-a.akamaihd.net/ugc/78...
2,15,1492.85,1480,906,1642945489,PSG.LGD,PSG.LGD,https://steamcdn-a.akamaihd.net/apps/dota2/ima...
3,8291895,1488.45,81,49,1642786657,Tundra Esports,Tundra,https://steamusercontent-a.akamaihd.net/ugc/17...
4,6209804,1472.72,471,335,1643027917,Royal Never Give Up,RNG,https://steamcdn-a.akamaihd.net/apps/dota2/ima...


In [43]:
# to load
leaders_df = pd.read_csv("data/leader_teams.csv")

### Collect info about matches

In [73]:
team_matches_df = pd.DataFrame()

with tqdm(total=leaders_df.shape[0]) as pbar:
    for index, team in leaders_df.iterrows():
        team_id = team['team_id']
        team_matches = pd.DataFrame(open_doto_api.get_team_matches(team_id))
        team_matches['team_id'] = team_id
        team_matches_df = pd.concat([team_matches_df, team_matches])
        pbar.update(1)

team_matches_df.to_csv('data/team_matches.csv', index=False)

100%|██████████| 150/150 [02:05<00:00,  1.20it/s]


In [120]:
team_matches_df = pd.read_csv("data/team_matches.csv")
team_matches_df["won"] = team_matches_df["radiant_win"] == team_matches_df["radiant"]
team_matches_df.head()

,match_id,radiant_win,radiant,duration,start_time,leagueid,league_name,cluster,opposing_team_id,opposing_team_name,opposing_team_logo,team_id,won
0,6388062021,True,True,2049,1642705587,13709,DPC EEU Division I Winter Tour - 2021/2022 pre...,273,1883502,Virtus.pro,https://steamcdn-a.akamaihd.net/apps/dota2/ima...,7119388,True
1,6387972688,True,True,1651,1642702235,13709,DPC EEU Division I Winter Tour - 2021/2022 pre...,273,1883502,Virtus.pro,https://steamcdn-a.akamaihd.net/apps/dota2/ima...,7119388,True
2,6380744310,True,True,2352,1642345042,13709,DPC EEU Division I Winter Tour - 2021/2022 pre...,273,7422789,HellRaisers,https://steamusercontent-a.akamaihd.net/ugc/18...,7119388,True
3,6380634943,True,True,1356,1642341748,13709,DPC EEU Division I Winter Tour - 2021/2022 pre...,274,7422789,HellRaisers,https://steamusercontent-a.akamaihd.net/ugc/18...,7119388,True
4,6368990292,True,True,2396,1641761422,13709,DPC EEU Division I Winter Tour - 2021/2022 pre...,274,36,Natus Vincere,https://steamcdn-a.akamaihd.net/apps/dota2/ima...,7119388,True


### Collect info about players

In [52]:
players_df = pd.DataFrame()
exclude_columns = ["is_current_team_member"]

for index, leader_team in leaders_df.iterrows():
    team_id = leader_team['team_id']
    players = pd.DataFrame.from_dict(open_doto_api.get_team_players(team_id))
    players = players[players["is_current_team_member"] == True]
    players["team_name"] = leader_team["name"]
    players["team_rating"] = leader_team["rating"]
    players["team_id"] = team_id
    players = players[[col for col in players.columns if col not in exclude_columns]]
    players_df = pd.concat([players_df, players])

players_df.to_csv('data/team_players.csv', index=False)

In [ ]:
players_df = pd.read_csv('data/team_players.csv')

In [53]:
players_stats_df = pd.DataFrame()

with tqdm(total=players_df.shape[0]) as pbar:
    for index, player in players_df.iterrows():
        player_info = pd.DataFrame(open_doto_api.get_player_info(player["account_id"]), index=[index])
        player_info.head()
        players_stats_df = pd.concat([players_stats_df, player_info])
        pbar.update(1)

players_stats_df.to_csv('data/player_stats.csv', index=False)

100%|██████████| 202/202 [03:09<00:00,  1.07it/s]


### Team matches investigation

In [114]:
# Let's filter teams without matches for last half of year
half_year = int((datetime.now() - timedelta(days=182)).timestamp())
active_teams = team_matches[["start_time", "team_id"]].groupby(["team_id"]).max().reset_index()
active_teams = active_teams[active_teams["start_time"] > half_year]
active_teams.head(10)
active_leaders_df = leaders_df[leaders_df["team_id"].isin(active_teams[TEAM_ID_FIELD])].copy()
", ".join(active_leaders_df["name"])

'Team Spirit, CyberBonch-1, PSG.LGD, Tundra Esports, Royal Never Give Up, BOOM Esports, Team Aster, T1, Virtus.pro, Fnatic, Undying, Team Secret, Evil Geniuses, Polaris Esports, Scenario, OG, OB.Neon, EHOME, Team SMG, Neptune Gaming, Xtreme\xa0 Gaming, Thunder Predator, Hellbear Smashers, Team Liquid, Team Tickles, INVICTUS GAMING, Quincy Crew, Team Flamingos, Nigma Galaxy, CIS Rejects, Entity, Galaxy Racer Esports, CREEPWAVE, Natus Vincere, Ragdoll, Ragdoll, PuckChamp, Infamous Uesports, Infamous U.esports., LBZS, Winstrike Team, Lilgun, beastcoast, Infinity, 4 Zoomers, Chicken Fighters, Motivate.Trust Gaming'

In [139]:
TEAM_NAME_TIMEZONE = {
    "Team Spirit": 'Europe/Moscow',
    "CyberBonch-1": 'Europe/Moscow',
    "PSG.LGD": 'Asia/Shanghai',
    "Tundra Esports": "Europe/London",
    "Royal Never Give Up": 'Asia/Shanghai',
    "BOOM Esports": "Asia/Jakarta",
    "Team Aster": "Asia/Shanghai",
    "T1": "Asia/Seoul",
    "Virtus.pro": "Europe/Moscow",
    "Fnatic": "Asia/Manila",
    "Undying": "America/Lima",
    "Team Secret": "Europe/Warsaw",
    "Evil Geniuses": "Europe/Helsinki",
   "Polaris Esports" : "Asia/Manila",
   "Scenario": "Asia/Tehran",
   "OG": "Europe/Moscow",
   "OB.Neon": "Asia/Manila",
   "EHOME": "Asia/Shanghai",
    "Team SMG": "Asia/Kuala_Lumpur",
    "Neptune Gaming": "Asia/Shanghai",
    "Xtreme\xa0 Gaming": "Asia/Shanghai",
    "Thunder Predator": "America/Lima",
    "Hellbear Smashers": "Europe/Copenhagen",
    "Team Liquid": "Europe/Stockholm",
    "Team Tickles": "Europe/Copenhagen",
    "INVICTUS GAMING": "Asia/Shanghai",
    "Quincy Crew": "America/New_York",
    "Team Flamingos": "Asia/Shanghai",
    "Nigma Galaxy": "Europe/Sofia",
    "CIS Rejects": 'Europe/Moscow',
    "Entity": "Europe/Vienna",
    "Galaxy Racer Esports": "Asia/Kuala_Lumpur",
    "CREEPWAVE": "Europe/Vienna",
    "Natus Vincere": "Europe/Kiev",
    "Ragdoll": "Asia/Kuala_Lumpur",
    "PuckChamp": "Europe/Moscow",
    "Infamous Uesports": "America/Lima",
    "Infamous U.esports.": "America/Lima",
    "LBZS": "Asia/Shanghai",
    "Winstrike Team": "Europe/Moscow",
    "Lilgun": "Asia/Ulaanbaatar",
    "beastcoast": "America/Lima",
    "Infinity": "America/Lima",
    "4 Zoomers": "America/New_York",
    "Chicken Fighters": "Europe/Stockholm",
    "Motivate.Trust Gaming": "Asia/Bangkok",
}

assert all(team_name in TEAM_NAME_TIMEZONE.keys() for team_name in active_leaders_df["name"]), f"Timezone for teams: 'f{[team_name for team_name in active_leaders_df['name'] not in TEAM_NAME_TIMEZONE.keys()]}' is missing!"

active_leaders_df["timezone"] = [pytz.timezone(TEAM_NAME_TIMEZONE[team_name]) for team_name in active_leaders_df["name"]]
matches_df = team_matches_df[["team_id", "won", "start_time", "duration"]].join(active_leaders_df[["team_id", "name", "timezone"]].set_index("team_id"), on="team_id").copy()
matches_df.apply(lambda x: pytz.utc.localize(datetime.fromtimestamp(x["start_time"])).astimezone(pytz.timezone(TEAM_NAME_TIMEZONE[x["name"]])), axis=1)
# matches_df.head(10)

,team_id,won,start_time,duration,name,timezone
0,7119388,True,1642705587,2049,Team Spirit,Europe/Moscow
1,7119388,True,1642702235,1651,Team Spirit,Europe/Moscow
2,7119388,True,1642345042,2352,Team Spirit,Europe/Moscow
3,7119388,True,1642341748,1356,Team Spirit,Europe/Moscow
4,7119388,True,1641761422,2396,Team Spirit,Europe/Moscow
5,7119388,True,1641756907,2464,Team Spirit,Europe/Moscow
6,7119388,False,1641750643,4329,Team Spirit,Europe/Moscow
7,7119388,True,1641484510,2684,Team Spirit,Europe/Moscow
8,7119388,True,1641481171,1559,Team Spirit,Europe/Moscow
9,7119388,False,1640028738,2548,Team Spirit,Europe/Moscow


In [62]:
df = leaders_df.join(players_df.set_index("team_id"), lsuffix="_team", rsuffix="_player", on="team_id").join(players_stats_df.set_index("account_id"), on="account_id")
df.head(10)

,team_id,rating,wins_team,losses,last_match_time,name_team,tag,logo_url,account_id,name_player,games_played,wins_player,team_name,team_rating,rank_tier,solo_mmr,leaderboard_rank
0,7119388,1550.29,216,173,1642705587,Team Spirit,TSpirit,https://steamusercontent-a.akamaihd.net/ugc/17...,321580662.0,YATOROGOD,223.0,150.0,Team Spirit,1550.29,80,5099.0,15
0,7119388,1550.29,216,173,1642705587,Team Spirit,TSpirit,https://steamusercontent-a.akamaihd.net/ugc/17...,113331514.0,Miposhka,223.0,150.0,Team Spirit,1550.29,80,6139.0,37
0,7119388,1550.29,216,173,1642705587,Team Spirit,TSpirit,https://steamusercontent-a.akamaihd.net/ugc/17...,431770905.0,TORONTOTOKYO,223.0,150.0,Team Spirit,1550.29,80,6139.0,40
0,7119388,1550.29,216,173,1642705587,Team Spirit,TSpirit,https://steamusercontent-a.akamaihd.net/ugc/17...,302214028.0,CoLLapse,223.0,150.0,Team Spirit,1550.29,80,6136.0,43
0,7119388,1550.29,216,173,1642705587,Team Spirit,TSpirit,https://steamusercontent-a.akamaihd.net/ugc/17...,256156323.0,Mira,206.0,140.0,Team Spirit,1550.29,80,6139.0,62
1,7412785,1520.12,267,1,1639928575,CyberBonch-1,CB,https://steamusercontent-a.akamaihd.net/ugc/78...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15,1492.85,1480,906,1642945489,PSG.LGD,PSG.LGD,https://steamcdn-a.akamaihd.net/apps/dota2/ima...,898754153.0,萧瑟,394.0,276.0,PSG.LGD,1492.85,80,7649.0,18
2,15,1492.85,1480,906,1642945489,PSG.LGD,PSG.LGD,https://steamcdn-a.akamaihd.net/apps/dota2/ima...,111114687.0,y`,379.0,264.0,PSG.LGD,1492.85,80,7362.0,94
2,15,1492.85,1480,906,1642945489,PSG.LGD,PSG.LGD,https://steamcdn-a.akamaihd.net/apps/dota2/ima...,157475523.0,XinQ,379.0,264.0,PSG.LGD,1492.85,80,7538.0,120
2,15,1492.85,1480,906,1642945489,PSG.LGD,PSG.LGD,https://steamcdn-a.akamaihd.net/apps/dota2/ima...,118134220.0,Faith_bian,379.0,264.0,PSG.LGD,1492.85,80,7351.0,21


In [67]:
df = df[df["solo_mmr"].notnull()]